# Pylops - stacking with missing operators

### Author: M.Ravasi

In this notebook, I will show how to efficiently set up some of the stacking operators in PyLops when some of the operators are equal zero (better than using `Zero` as this does unefficient memory copies)

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib inline

#import warnings
#warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

from pylops.utils                      import dottest
from pylops.utils.wavelets             import *
from pylops.utils.seismicevents        import *
from pylops.basicoperators             import *
from pylops.signalprocessing           import *
from pylops.waveeqprocessing.mdd       import *
from pylops.optimization.leastsquares  import *
from pylops.optimization.sparsity import irls as irlspylops
from pylops.optimization.sparsity import fista, spgl1

## Vertical stacking of operators

In [2]:
A= np.zeros((11,21))
A[5,10]=1

D2hop = SecondDerivative(dims=[11,21], axis=1, dtype='float64')
Zop = Zero(500000, 11 * 21)

Vstackop = VStack([D2hop, Zop])
dottest(Vstackop, 11*21 + 500000,11*21)
B= Vstackop @ A.ravel()

Vstackop = VStack([D2hop, *[Zop] * 5])
dottest(Vstackop, 11*21 + 5 * 500000, 11*21)
B= Vstackop @ A.ravel()


In [3]:
Vstackop1 = VStack([D2hop, *[500000] * 5])
dottest(Vstackop1, 11*21 + 5 * 500000, 11*21)
B1 = Vstackop1 @ A.ravel()

np.allclose(B, B1)

True

In [4]:
%timeit -n 10 -r 2 Vstackop @ A.ravel()
%timeit -n 10 -r 2 Vstackop1 @ A.ravel()

%timeit -n 10 -r 2 Vstackop.H @ B.ravel()
%timeit -n 10 -r 2 Vstackop1.H @ B.ravel()

3.18 ms ± 297 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
1.38 ms ± 103 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
29.3 μs ± 4.36 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
19.6 μs ± 1.32 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


## Horizontal stacking of operators

In [5]:
A = np.arange(11*21 + 5 * 500000)

D2hop = SecondDerivative(dims =[11,21], axis=1, dtype='float64')
Zop = Zero(11 * 21, 500000)

Hstackop = HStack([D2hop, *[Zop] * 5])
dottest(Hstackop, 11*21, 11*21 + 5 * 500000)
B= Hstackop @ A.ravel()


In [6]:
Hstackop1 = HStack([D2hop, *[500000] * 5])
dottest(Hstackop, 11*21, 11*21 + 5 * 500000)
B1 = Hstackop1 @ A.ravel()

np.allclose(B, B1.ravel())

True

In [7]:
%timeit -n 10 -r 2 Hstackop @ A.ravel()
%timeit -n 10 -r 2 Hstackop1 @ A.ravel()

%timeit -n 10 -r 2 Hstackop.H @ B.ravel()
%timeit -n 10 -r 2 Hstackop1.H @ B.ravel()

The slowest run took 4.29 times longer than the fastest. This could mean that an intermediate result is being cached.
40.8 μs ± 25.4 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
35.7 μs ± 3.28 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
3.44 ms ± 233 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
1.38 ms ± 23.7 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


## Block matrix

In [8]:
nbl = 50000
bl = Block([[Identity(nbl), Zero(nbl),       Zero(nbl)],
            [Zero(nbl),     2*Identity(nbl), Zero(nbl)],
            [Zero(nbl),     Zero(nbl),       -1*Identity(nbl)]])
print(bl)
print(bl*np.ones(3*nbl))

bl1 = Block([[Identity(nbl), nbl,               nbl],
             [nbl,           2*Identity(nbl),   nbl],
             [nbl,           nbl,              -1*Identity(nbl)]])
print(bl1)
print(bl1*np.ones(3*nbl))


<150000x150000 Block with dtype=float64>
[ 1.  1.  1. ... -1. -1. -1.]
<150000x150000 Block with dtype=float64>
[ 1.  1.  1. ... -1. -1. -1.]


In [9]:
%timeit -n 10 -r 2 bl @ np.ones(3*nbl)
%timeit -n 10 -r 2 bl1 @ np.ones(3*nbl)

%timeit -n 10 -r 2 bl.H @ np.ones(3*nbl)
%timeit -n 10 -r 2 bl1.H @ np.ones(3*nbl)

491 μs ± 35.3 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
526 μs ± 73.7 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
661 μs ± 4.25 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
334 μs ± 19.3 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
